In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 14.4 MB/s 
     |████████████████████████████████| 596 kB 56.6 MB/s 
     |████████████████████████████████| 6.6 MB 56.2 MB/s 
     |████████████████████████████████| 86 kB 6.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.8 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=09ef393a9aaa1c681423377e33cdeedc419717d366c5c990d7ffef62a9641170
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
import pickle
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf
import pdb


def load_pickle(data_name):
    with open('/content/drive/MyDrive/2022-1-자연어처리/data/preprocessed/v_final/{}.pickle'.format(data_name),'rb') as f:
        data = pickle.load(f)
    
    return data

In [ ]:
# set data name
data_type = 'total'

# model name 
model_name = 'bert_{}_test1'.format(data_type)

pretrain_model = "klue/bert-base"
checkpoint_dir = '/content/drive/MyDrive/2022-1-자연어처리/code/NER_KoBERT/checkpoints'
os.makedirs(os.path.join(checkpoint_dir, model_name), exist_ok=True)

train_data_name = 'train_{}'.format(data_type)
test_data_name = 'test_{}'.format(data_type)

train_ner_df = load_pickle(train_data_name)
test_ner_df = load_pickle(test_data_name)

# split sentence and label sequence with ' ' space 
if not (train_data_name in ['train_naver', 'train_total']):
    train_data_sentence = [example[0] for example in train_ner_df]
    test_data_sentence = [example[0] for example in test_ner_df]

    train_data_pos = [example[1] for example in train_ner_df]
    test_data_pos = [example[1] for example in test_ner_df]

    train_data_label = [example[2] for example in train_ner_df]
    test_data_label = [example[2] for example in test_ner_df]
else:
    train_data_sentence = [example[0] for example in train_ner_df]
    test_data_sentence = [example[0] for example in test_ner_df]

    train_data_label = [example[1] for example in train_ner_df]
    test_data_label = [example[1] for example in test_ner_df]

labels = [label.strip() for label in open('/content/drive/MyDrive/2022-1-자연어처리/code/NER_KoBERT/ner_{}_label.txt'.format(data_type), 'r', encoding='utf-8')]
print('개체명 태깅 정보 :', labels)

개체명 태깅 정보 : ['O', 'PER_B', 'PER_I', 'COM_B', 'COM_I', 'LOC_B', 'LOC_I', 'POS_B', 'POS_I', 'NUM_B', 'NUM_I']


In [ ]:
# make dictionary for label (tag => index) (index => label)
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}
tag_size = len(tag_to_index)
print('개체명 태깅 정보의 개수 :',tag_size)
print(tag_to_index)

개체명 태깅 정보의 개수 : 11
{'O': 0, 'PER_B': 1, 'PER_I': 2, 'COM_B': 3, 'COM_I': 4, 'LOC_B': 5, 'LOC_I': 6, 'POS_B': 7, 'POS_I': 8, 'NUM_B': 9, 'NUM_I': 10}


In [ ]:
tokenizer = BertTokenizer.from_pretrained(pretrain_model)

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
''' convert_examples_to_features
    * Args 
        example : List with String (sentence_num, sentence_len), sentence word str 
        labels : List with String (sentence_num, sentence_len), sentence tag str
        max_seq_len : Int, max sequence length, all output size will be same with max sequence length 
        tokenizer : tokenizer from transformers
        pad_token_id_for_segment : Int, pad token index for segment
        pad_token_id_for_label : Int, pad token index for label
    
    * Outputs
        x : tuple with list ((num_sentence, max_seq_len),(num_sentence, max_seq_len),(num_sentence, max_seq_len)), It has three type's data (input_id, attention_mask, token_type_ids), It's a tokenized feature
        y : list, label list, It pair with the tokenized feature
'''

def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    # load token info from tokenizer 
    # ex) if the tokenizer ("klue/bert-base") => str
    cls_token = tokenizer.cls_token # [CLS]
    sep_token = tokenizer.sep_token # [SEP]
    pad_token_id = tokenizer.pad_token_id # [PAD]

    # output list
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    # load sentence data
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        tokens = []
        labels_ids = []

        # load subsentence data
        for one_word, label_token in zip(example, label):
            # tokenize to the subword unit automatically using tokenizer
            subword_tokens = tokenizer.tokenize(one_word)
            # pdb.set_trace()
            tokens.extend(subword_tokens)
            labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1)) # [word_label, pad_label1, pad_label2 ...]


        # limitation of the sentence len
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

        # add seperate token (one sentence finished) if we have two sentence it will be used
        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label]

        # add classification tokens (before the sentence)
        # tokens = [cls_token] + tokens
        # labels_ids = [pad_token_id_for_label] + labels_ids


        # tokenize to index => maybe the vector embedding
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        padding_count = max_seq_len - len(input_id)
        
        # attention masking step, 1.count input_id size, 2.make attention mask [1,1,1,...] 3. padding the atten mask that not in the input_id [1,1,1...,0,0..]
        attention_mask = [1] * len(input_id)
        attention_mask = attention_mask + ([0] * padding_count)

        # padding step, 1.count pad size 2.padding to input_id
        input_id = input_id + ([pad_token_id] * padding_count)


        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label = labels_ids + ([pad_token_id_for_label] * padding_count)

        # check the length, all output(input_id, attention_mask, token_type_id, label) are the same with max_seq_len
        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
# prepare the data to train the model (wrapping as the dataset)
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 19428/19428 [00:26<00:00, 722.97it/s] 


In [ ]:
''' TFBertForTokenClassification : tensorflow model wrapped by tf.keras.Model

    * Arch
        pretrained BERT - dense layer for classification

'''
class TFBertForTokenClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForTokenClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        all_output = outputs[0]
        prediction = self.classifier(all_output)

        return prediction

In [ ]:
''' compute_loss : computation of loss using tf.keras.lossess
    
    the loss only computed if not the label -100(pad)

'''
def compute_loss(labels, logits):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
                from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
    active_loss = tf.reshape(labels, (-1,)) != -100
    reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
    labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
    
    return loss_fn(labels, reduced_logits)

In [ ]:
# TPU 작동을 위한 코드
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

# strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
# with strategy.scope():
model = TFBertForTokenClassification(pretrain_model, num_labels=tag_size)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=compute_loss, run_eagerly=False)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test

    def sequences_to_tags(self, label_ids, pred_ids):
      label_list = []
      pred_list = []

      for i in range(0, len(label_ids)):
        label_tag = []
        pred_tag = []

        for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
          if label_index != -100:
            label_tag.append(index_to_tag[label_index])
            pred_tag.append(index_to_tag[pred_index])
        
        label_list.append(label_tag)
        pred_list.append(pred_tag)

      return label_list, pred_list

    def on_epoch_end(self, epoch, logs={}):

      y_predicted = self.model.predict(self.X_test)
      y_predicted = np.argmax(y_predicted, axis = 2)

      label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

      score = f1_score(label_list, pred_list, suffix=True)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(label_list, pred_list, suffix=True))

In [ ]:
# callbacks
f1_score_report = F1score(X_test, y_test)

# 모델의 가중치를 저장하는 콜백 만들기
checkpoint_path = checkpoint_dir + "/{}/saved_wights".format(model_name)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

In [ ]:
model.fit(
    X_train, y_train, epochs=10, batch_size=32,
    validation_split=0.1,
    callbacks = [f1_score_report])

Epoch 1/10
4273/4273 [==============================] - ETA: 0s - loss: 0.0459

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: COM_B seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: PER_B seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: PER_I seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: LOC_B seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: NUM_B seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:167:

 - f1: 91.21
              precision    recall  f1-score   support

        COM_       0.90      0.94      0.92     14638
        LOC_       0.91      0.90      0.91      5995
        NUM_       0.83      0.93      0.88      8601
        PER_       0.95      0.95      0.95     12032
        POS_       0.94      0.80      0.86      5146

   micro avg       0.90      0.92      0.91     46412
   macro avg       0.91      0.91      0.90     46412
weighted avg       0.90      0.92      0.91     46412

4273/4273 [==============================] - 1332s 307ms/step - loss: 0.0459 - val_loss: 0.1041
Epoch 2/10
4273/4273 [==============================] - ETA: 0s - loss: 0.0268 - f1: 92.48
              precision    recall  f1-score   support

        COM_       0.92      0.95      0.93     14638
        LOC_       0.90      0.90      0.90      5995
        NUM_       0.88      0.93      0.91      8601
        PER_       0.95      0.95      0.95     12032
        POS_       0.93      0.87      0

In [ ]:
model.save_weights(checkpoint_path)

In [ ]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]

        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask


        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [ ]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

In [ ]:
def ner_prediction(examples, max_seq_len, tokenizer):
  examples = [sent.split() for sent in examples]
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted, axis = 2)

  pred_list = []
  result_list = []

  for i in range(0, len(label_masks)):
    pred_tag = []
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])

    pred_list.append(pred_tag)

  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)

  return result_list